## Ingest lap_time folder
Step 1- Read the csv file using the spark dataframe reader API

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType, StructField, TimestampType, DateType, FloatType
from pyspark.sql.functions import col, struct,current_timestamp , concat, lit

In [0]:
lap_times_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                       StructField("driverId", IntegerType(), True), 
                                       StructField("lap", IntegerType(), True), 
                                       StructField("position", IntegerType(), True),
                                       StructField("time", StringType(), True), 
                                       StructField("milliseconds", IntegerType(), True)])

In [0]:
# Load the JSON file
lap_times_df = spark.read\
    .schema(lap_times_schema)\
    .csv(f"{raw_folder_path}/{v_file_date}/lap_times")

In [0]:
display(lap_times_df)

raceId,driverId,lap,position,time,milliseconds
1053,830,1,1,1:38.603,98603
1053,830,2,1,2:29.163,149163
1053,830,3,1,2:23.247,143247
1053,830,4,1,2:20.332,140332
1053,830,5,1,2:25.691,145691
1053,830,6,1,2:20.804,140804
1053,830,7,1,1:36.303,96303
1053,830,8,1,1:32.925,92925
1053,830,9,1,1:30.953,90953
1053,830,10,1,1:30.130,90130


In [0]:
lap_times_df.count()

Out[77]: 1124

#### Step 2 - Rename cloumns and add new columns
driverId and raceId

In [0]:
lap_times_final_df = lap_times_df.withColumnRenamed("driverId", "driver_id")\
  .withColumnRenamed("raceId", "race_id")\
   .withColumn("data_source", lit(v_data_source)) 
                                           

In [0]:
lap_times_final_df = add_ingestion_date(lap_times_final_df)

In [0]:
display(lap_times_final_df)

race_id,driver_id,lap,position,time,milliseconds,data_source,ingestion_date
1053,830,1,1,1:38.603,98603,testing,2025-04-10T22:37:44.617+0000
1053,830,2,1,2:29.163,149163,testing,2025-04-10T22:37:44.617+0000
1053,830,3,1,2:23.247,143247,testing,2025-04-10T22:37:44.617+0000
1053,830,4,1,2:20.332,140332,testing,2025-04-10T22:37:44.617+0000
1053,830,5,1,2:25.691,145691,testing,2025-04-10T22:37:44.617+0000
1053,830,6,1,2:20.804,140804,testing,2025-04-10T22:37:44.617+0000
1053,830,7,1,1:36.303,96303,testing,2025-04-10T22:37:44.617+0000
1053,830,8,1,1:32.925,92925,testing,2025-04-10T22:37:44.617+0000
1053,830,9,1,1:30.953,90953,testing,2025-04-10T22:37:44.617+0000
1053,830,10,1,1:30.130,90130,testing,2025-04-10T22:37:44.617+0000



#### Step 3 - Write to output to processed container in parquet format

In [0]:
#lap_times_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/lap_times")
#lap_times_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.lap_times")

In [0]:
#overwrite_partition(lap_times_final_df, "f1_processed", "lap_times", "race_id")

In [0]:
merge_condition ="tgt.race_id =src.race_id AND tgt.driver_id = src.driver_id AND tgt.lap = src.lap AND tgt.race_id = src.race_id"
merge_delta_data (lap_times_final_df,"f1_processed", "lap_times",processed_folder_path,merge_condition,"race_id")

In [0]:
%fs
ls /mnt/formula1dl2025practice/processed/lap_times

path,name,size,modificationTime
dbfs:/mnt/formula1dl2025practice/processed/lap_times/_delta_log/,_delta_log/,0,1744324545000
dbfs:/mnt/formula1dl2025practice/processed/lap_times/race_id=1/,race_id=1/,0,1744324546000
dbfs:/mnt/formula1dl2025practice/processed/lap_times/race_id=10/,race_id=10/,0,1744324547000
dbfs:/mnt/formula1dl2025practice/processed/lap_times/race_id=100/,race_id=100/,0,1744324550000
dbfs:/mnt/formula1dl2025practice/processed/lap_times/race_id=1000/,race_id=1000/,0,1744324560000
dbfs:/mnt/formula1dl2025practice/processed/lap_times/race_id=1001/,race_id=1001/,0,1744324560000
dbfs:/mnt/formula1dl2025practice/processed/lap_times/race_id=1002/,race_id=1002/,0,1744324561000
dbfs:/mnt/formula1dl2025practice/processed/lap_times/race_id=1003/,race_id=1003/,0,1744324561000
dbfs:/mnt/formula1dl2025practice/processed/lap_times/race_id=1004/,race_id=1004/,0,1744324561000
dbfs:/mnt/formula1dl2025practice/processed/lap_times/race_id=1005/,race_id=1005/,0,1744324561000


In [0]:
#display(spark.read.parquet(f"{processed_folder_path}/lap_times"))

In [0]:
dbutils.notebook.exit("Success")

In [0]:
%sql

SELECT race_id, count(1)
FROM f1_processed.lap_times
GROUP BY race_id
ORDER BY race_id DESC